<a href="https://colab.research.google.com/github/shackste/galaxy-generator/blob/main/ACGAN_CVAE_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PyTorch implementation of combined Conditional Variable Auto Encoder (CVAE) and Auxiliary-Classifier Generative Adversarial Network (ACGAN)

continuation of work by Mohamad Dia

# Environment Setup

## Modules

In [ ]:
!pip install torchviz

In [ ]:
import os
import imageio
from datetime import datetime
from pdb import set_trace

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import Image, display
import torch
from torch import rand, cat, tensor, log, square, sum, mean, add, from_numpy
from torch.nn import Module, Sequential, \
                     Conv2d, ConvTranspose2d, Linear, \
                     ReLU, LeakyReLU, Sigmoid, Softmax, Softplus, \
                     BatchNorm1d, BatchNorm2d, Flatten, ZeroPad2d, \
                     MSELoss, BCELoss, CrossEntropyLoss                    
from torch.optim import Adam
from torchsummary import summary
from torchviz import make_dot

## File system

In [ ]:
# mount google drive
from google.colab import drive
drive.mount("/drive")

root = "/drive/MyDrive/FHNW/galaxy_generator/"

## the following files need to be downloaded to your google drive in root folder defined above
## download from https://www.kaggle.com/c/galaxy-zoo-the-galaxy-challenge/data
file_galaxy_images = root + "galaxyzoo_data_cropped_nonnormalized.npy"
file_galaxy_labels = root + "training_solutions_rev1.csv"

folder_results = root + "results_pytorch/"

if not os.path.exists(folder_results):
    os.mkdir(folder_results)


Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


## Parameter

In [ ]:
# Input
image_dim = 64
colors_dim = 3
labels_dim = 3
input_size = (colors_dim,image_dim,image_dim)

# Encoder/Decoder
latent_dim = 8
decoder_dim = latent_dim  # differs from latent_dim if PCA applied before decoder

# General
learning_rate = 0.0002
betas = (0.5,0.999)  ## 0.999 is default beta2 in tensorflow
optimizer = Adam
negative_slope = 0.2 # for LeakyReLU
momentum = 0.99 # for BatchNorm

# Loss weights
alpha = 1 # switch VAE (1) / AE (0)
beta = 1 # weight for KL-loss
zeta = 0.5 # weight for MSE-loss
delta = 1 # weight for class-loss
gamma = 1024 # weight for learned-metric-loss (https://arxiv.org/pdf/1512.09300.pdf)


# helpful functions

In [ ]:
def summarize(network, input_size=input_size, graph=False, labels=False):
    net = network()
    net = net.cuda()
    print("input size", input_size)
    input_dummy = rand(3, *input_size).cuda()
    if labels:
        summary(net, input_size)
        labels_dummy = rand(labels_dim).cuda()
        y = net(input_dummy, labels_dummy)
    else:
        summary(net, input_size)
        y = net(input_dummy)
    if graph:
        display(make_dot(y, params=dict(list(net.named_parameters()))))

In [ ]:
## create module to reshape in Sequential
class Reshape(Module):
    def __init__(self, *shape):
        super(Reshape, self).__init__()
        self.shape = shape
    def forward(self, input):
        return input.view(input.shape[0], *self.shape)

In [ ]:
## create module to check shape is Sequential
class PrintShape(Module):
    def __init__(self):
        super(PrintShape, self).__init__()
    def forward(self, input):
        print(input.shape)
        return input

In [ ]:
def same_padding(*, kernel_size=1, stride=1):
    return (kernel_size - stride)//2 + 1

In [ ]:
def write_generated_galaxy_image(*, image=None, filename=None):
    assert image is not None, "provide generated galaxy image"
    assert filename is not None, "provide filename"
    if not image.shape[2] == 3:
        image = np.rollaxis(image, 0, 3)
    imageio.imwrite(folder_results+filename, image)


def write_generated_galaxy_images_iteration(*, iteration=None, images=None):
    assert type(iteration) is int, "provide iteration as integer"
    assert images is not None, "provide generated galaxy images"

    w, h = 8, 8  ## !! find where these come from
    d = image_dim

    flat_image = np.empty((3, w*d, h*d))
    k = 0
    for iw in range(w):
        for ih in range(h):
            flat_image[:,iw*d:(iw+1)*d, ih*d:(ih+1)*d] = images[k]
            k += 1
    write_generated_galaxy_image(image=flat_image, filename=f"samples_iter{iteration}.png")

In [ ]:
def return_batch(sample, i, size):
    """ from sample return the i'th batch if size """
    return sample[i*size : (i+1)*size]

# Neural Networks

In [ ]:
class NeuralNetwork(Module):
    """ base class with convenient procedures used by all NN"""
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.parameter_file = f"{folder_results}parameter_state_dict_{self._get_name()}.pth"
        # self.cuda() ## all NN shall run on cuda ### doesnt seem to work
    
    def save(self):
        torch.save(self.state_dict(), self.parameter_file)

    def load(self):
        self.load_state_dict(torch.load(self.parameter_file))
        self.eval()


## Discriminator

### Discriminator 1

In [ ]:
class Discriminator1(NeuralNetwork):
    def __init__(self):
        super(Discriminator1, self).__init__()
        kernel_size = 3
        stride = 2
        padding = (kernel_size - 1) // 2
        self.conv1 = Sequential(
            Conv2d(colors_dim, 8, kernel_size=kernel_size, stride=stride, padding=padding),
            ReLU(),
        )
        self.conv2 = Sequential(
            Conv2d(8, 16, kernel_size=kernel_size, stride=stride, padding=padding),
            BatchNorm2d(16),
            ReLU(),
        )
        self.conv3 = Sequential(
            Conv2d(16, 32, kernel_size=kernel_size, stride=stride, padding=padding),
            ReLU(),
        )
        self.conv4 = Sequential(
            Conv2d(32, 64, kernel_size=kernel_size, stride=stride, padding=padding),
            ReLU(),
        )
        self.dense1 = Sequential(
            Flatten(),
            Linear(1024,256),
            ReLU(),
        )
        self.dense2 = Sequential(
            Linear(256,1),
            Sigmoid(),
        )
        self.optimizer = optimizer(self.parameters(), lr=learning_rate, betas=betas)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.dense1(x)
        x = self.dense2(x)
        return x

In [ ]:
summarize(Discriminator1, graph=False)

input size (3, 64, 64)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 32, 32]             224
              ReLU-2            [-1, 8, 32, 32]               0
            Conv2d-3           [-1, 16, 16, 16]           1,168
       BatchNorm2d-4           [-1, 16, 16, 16]              32
              ReLU-5           [-1, 16, 16, 16]               0
            Conv2d-6             [-1, 32, 8, 8]           4,640
              ReLU-7             [-1, 32, 8, 8]               0
            Conv2d-8             [-1, 64, 4, 4]          18,496
              ReLU-9             [-1, 64, 4, 4]               0
          Flatten-10                 [-1, 1024]               0
           Linear-11                  [-1, 256]         262,400
             ReLU-12                  [-1, 256]               0
           Linear-13                    [-1, 1]             257
          Sigmoi

### Discriminator 3

In [ ]:
class Discriminator3(NeuralNetwork):
    def __init__(self):
        super(Discriminator3, self).__init__()
        kernel_size = 5
        stride = 2
        padding = (kernel_size - 1) // 2

        self.conv0 = Sequential(
            Conv2d(colors_dim, 32, kernel_size=1, stride=1),
            LeakyReLU(negative_slope=negative_slope),
        )
        self.conv1 = Sequential(
            Conv2d(32, 64, kernel_size=kernel_size, stride=stride, padding=padding),
            BatchNorm2d(64, momentum=momentum),
            LeakyReLU(negative_slope=negative_slope),
        )
        self.conv2 = Sequential(
            Conv2d(64, 128, kernel_size=kernel_size, stride=stride, padding=padding),
            BatchNorm2d(128, momentum=momentum),
            LeakyReLU(negative_slope=negative_slope),
        )
        self.dense1 = Sequential(
            Flatten(),
            Linear(32768,1024),
            BatchNorm1d(1024, momentum=momentum),
            LeakyReLU(negative_slope=negative_slope),
        )
        self.dense2 = Sequential(
            Linear(1024,1),
            Sigmoid(),
        )
        self.optimizer = optimizer(self.parameters(), lr=learning_rate, betas=betas)


    def forward(self, x):
        x = self.conv0(x)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.dense1(x)
        x = self.dense2(x)
        return x

In [ ]:
summarize(Discriminator3, graph=False)

input size (3, 64, 64)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 64, 64]             128
         LeakyReLU-2           [-1, 32, 64, 64]               0
            Conv2d-3           [-1, 64, 32, 32]          51,264
       BatchNorm2d-4           [-1, 64, 32, 32]             128
         LeakyReLU-5           [-1, 64, 32, 32]               0
            Conv2d-6          [-1, 128, 16, 16]         204,928
       BatchNorm2d-7          [-1, 128, 16, 16]             256
         LeakyReLU-8          [-1, 128, 16, 16]               0
           Flatten-9                [-1, 32768]               0
           Linear-10                 [-1, 1024]      33,555,456
      BatchNorm1d-11                 [-1, 1024]           2,048
        LeakyReLU-12                 [-1, 1024]               0
           Linear-13                    [-1, 1]           1,025
          Sigmoi

### Discriminator 4

In [ ]:
class Discriminator4(NeuralNetwork):
    def __init__(self):
        super(Discriminator4, self).__init__()
        kernel_size = 3
        stride = 2
        padding = (kernel_size - 1) // 2
        self.conv0 = Sequential(
            Conv2d(colors_dim, 16, kernel_size=1, stride=1),
            LeakyReLU(negative_slope=negative_slope),
        )
        self.conv1 = Sequential(
            Conv2d(16, 32, kernel_size=kernel_size, stride=stride, padding=padding),
            BatchNorm2d(32, momentum=momentum),
            LeakyReLU(negative_slope=negative_slope)
        )
        self.conv2 = Sequential(
            Conv2d(32, 64, kernel_size=kernel_size, stride=stride, padding=padding),
            BatchNorm2d(64, momentum=momentum),
            LeakyReLU(negative_slope=negative_slope)
        )
        self.conv3 = Sequential(
            Conv2d(64, 128, kernel_size=kernel_size, stride=stride, padding=padding),
            BatchNorm2d(128, momentum=momentum),
            LeakyReLU(negative_slope=negative_slope)
        )
        self.dense1 = Sequential(
            Flatten(),
            Linear(8192, 1024),
            LeakyReLU(negative_slope=negative_slope)
        )   ## outputs metric

        ## the following take metric as input
        self.dense2_1 = Sequential(
            Linear(1024,1),
            Sigmoid(),
        )   ## outputs true/fake
        self.dense2_2 = Sequential(
            Linear(1024, labels_dim),
            Softmax(dim=1),
        )   ## outputs labels
        self.optimizer = optimizer(self.parameters(), lr=learning_rate, betas=betas)


    def forward(self, images):
        x = self.conv0(images)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        metric = self.dense1(x)
        true_fake = self.dense2_1(metric)
        labels = self.dense2_2(metric)
        return cat((true_fake, labels, metric), dim=1)


In [ ]:
summarize(Discriminator4, graph=False)

input size (3, 64, 64)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 64, 64]              64
         LeakyReLU-2           [-1, 16, 64, 64]               0
            Conv2d-3           [-1, 32, 32, 32]           4,640
       BatchNorm2d-4           [-1, 32, 32, 32]              64
         LeakyReLU-5           [-1, 32, 32, 32]               0
            Conv2d-6           [-1, 64, 16, 16]          18,496
       BatchNorm2d-7           [-1, 64, 16, 16]             128
         LeakyReLU-8           [-1, 64, 16, 16]               0
            Conv2d-9            [-1, 128, 8, 8]          73,856
      BatchNorm2d-10            [-1, 128, 8, 8]             256
        LeakyReLU-11            [-1, 128, 8, 8]               0
          Flatten-12                 [-1, 8192]               0
           Linear-13                 [-1, 1024]       8,389,632
        LeakyReL

## Encoder

### Encoder 1
basic convolutional network

In [ ]:
class Encoder1(NeuralNetwork):
    def __init__(self):
        super(Encoder1, self).__init__()
        kernel_size = 3
        stride = 2
        padding = (kernel_size - 1) // 2
        self.conv1 = Sequential(
            Conv2d(colors_dim, 8, kernel_size=kernel_size, stride=stride, padding=padding),
            ReLU(),
        )
        self.conv2 = Sequential(
            Conv2d(8, 16, kernel_size=kernel_size, stride=stride, padding=padding),
            BatchNorm2d(16),
            ReLU(),
        )
        self.conv3 = Sequential(
            Conv2d(16, 32, kernel_size=kernel_size, stride=stride, padding=padding),
            ReLU(),
        )
        self.conv4 = Sequential(
            Conv2d(32, 64, kernel_size=kernel_size, stride=stride, padding=padding),
            ReLU(),
        )
        self.dense1 = Sequential(
            Flatten(),
            Linear(1024, 256),
            ReLU(),
        )
        
        ## the following take the same input
        self.dense_z_mu = Linear(256, latent_dim)
        if alpha:
            self.dense_z_std = Sequential(
                Linear(256, latent_dim),
                Softplus(),
            )
        self.optimizer = optimizer(self.parameters(), lr=learning_rate, betas=betas)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.dense1(x)
        z_mu = self.dense_z_mu(x)
        if not alpha:
            return z_mu
        z_std = self.dense_z_std(x)
        return cat((z_mu, z_std), dim=1)

In [ ]:
summarize(Encoder1, graph=False)

input size (3, 64, 64)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 32, 32]             224
              ReLU-2            [-1, 8, 32, 32]               0
            Conv2d-3           [-1, 16, 16, 16]           1,168
       BatchNorm2d-4           [-1, 16, 16, 16]              32
              ReLU-5           [-1, 16, 16, 16]               0
            Conv2d-6             [-1, 32, 8, 8]           4,640
              ReLU-7             [-1, 32, 8, 8]               0
            Conv2d-8             [-1, 64, 4, 4]          18,496
              ReLU-9             [-1, 64, 4, 4]               0
          Flatten-10                 [-1, 1024]               0
           Linear-11                  [-1, 256]         262,400
             ReLU-12                  [-1, 256]               0
           Linear-13                    [-1, 8]           2,056
           Linea

### Encoder 2
inception convolutional network

In [ ]:
class Encoder2(NeuralNetwork):
    """ inception convolutional network """
    def __init__(self):
        super(Encoder2, self).__init__()
        kernel_size = 3
        stride = 2
        padding = (kernel_size - 1) // 2

        # Inception 1
        self.inc1_1 = Sequential(
            Conv2d(colors_dim, 4, kernel_size=1, stride=1),
            ReLU(),
        )
        self.inc1_2 = Sequential(
            Conv2d(colors_dim, 4, kernel_size=kernel_size, stride=1, padding=padding),
            ReLU(),
        )
        self.conv1 = Sequential(
            Conv2d(8, 8, kernel_size=kernel_size, stride=stride, padding=padding),
            ReLU(),
        )

        # Inception 2
        self.inc2_1 = Sequential(
            Conv2d(8, 8, kernel_size=1, stride=1),
            ReLU(),
        )
        self.inc2_2 = Sequential(
            Conv2d(8, 8, kernel_size=kernel_size, stride=1, padding=padding),
            ReLU(),
        )
        self.conv2 = Sequential(
            Conv2d(16, 16, kernel_size=kernel_size, stride=stride, padding=padding),
            BatchNorm2d(16),
            ReLU(),
        )

        # Inception 3
        self.inc3_1 = Sequential(
            Conv2d(16, 16, kernel_size=1, stride=1),
            ReLU(),
        )
        self.inc3_2 = Sequential(
            Conv2d(16, 16, kernel_size=kernel_size, stride=1, padding=padding),
            ReLU(),
        )
        self.conv3 = Sequential(
            Conv2d(32, 32, kernel_size=kernel_size, stride=stride, padding=padding),
            ReLU(),
        )

        # Inception 4
        self.inc4_1 = Sequential(
            Conv2d(32, 32, kernel_size=1, stride=1),
            ReLU(),
        )
        self.inc4_2 = Sequential(
            Conv2d(32, 32, kernel_size=kernel_size, stride=1, padding=padding),
            ReLU(),
        )
        self.conv4 = Sequential(
            Conv2d(64, 64, kernel_size=kernel_size, stride=stride, padding=padding),
            ReLU(),
        )

        self.dense1 = Sequential(
            Flatten(),
            Linear(1024, 256),
            ReLU(),
        )

        ## the following take the same input from dense1
        self.dense_z_mu = Linear(256, latent_dim)
        if alpha:
            self.dense_z_std = Sequential(
                Linear(256, latent_dim),
                Softplus(),
            )
        self.optimizer = optimizer(self.parameters(), lr=learning_rate, betas=betas)

    def forward(self, x):
        inc1 = self.inc1_1(x)
        inc2 = self.inc1_2(x)
        x = self.conv1(cat((inc1,inc2), dim=1))
        inc1 = self.inc2_1(x)
        inc2 = self.inc2_2(x)
        x = self.conv2(cat((inc1,inc2), dim=1))
        inc1 = self.inc3_1(x)
        inc2 = self.inc3_2(x)
        x = self.conv3(cat((inc1,inc2), dim=1))
        inc1 = self.inc4_1(x)
        inc2 = self.inc4_2(x)
        x = self.conv4(cat((inc1,inc2), dim=1))
        x = self.dense1(x)
        z_mu = self.dense_z_mu(x)
        if not alpha:
            return z_mu
        z_std = self.dense_z_std(x)
        return cat((z_mu, z_std), dim=1)

        

In [ ]:
summarize(Encoder2, graph=False)

input size (3, 64, 64)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 4, 64, 64]              16
              ReLU-2            [-1, 4, 64, 64]               0
            Conv2d-3            [-1, 4, 64, 64]             112
              ReLU-4            [-1, 4, 64, 64]               0
            Conv2d-5            [-1, 8, 32, 32]             584
              ReLU-6            [-1, 8, 32, 32]               0
            Conv2d-7            [-1, 8, 32, 32]              72
              ReLU-8            [-1, 8, 32, 32]               0
            Conv2d-9            [-1, 8, 32, 32]             584
             ReLU-10            [-1, 8, 32, 32]               0
           Conv2d-11           [-1, 16, 16, 16]           2,320
      BatchNorm2d-12           [-1, 16, 16, 16]              32
             ReLU-13           [-1, 16, 16, 16]               0
           Conv2

### Encoder 3
BatchNormed convolutional network with LeakyReLU and kernel=5

In [ ]:
class Encoder3(NeuralNetwork):
    """ convolutional network with leaky ReLU """
    def __init__(self):
        super(Encoder3, self).__init__()
        kernel_size = 5
        stride = 2
        padding = (kernel_size - 1) // 2

        self.conv0 = Sequential(
            Conv2d(colors_dim, 32, kernel_size=1, stride=1),
            LeakyReLU(negative_slope=negative_slope),
        )
        self.conv1 = Sequential(
            Conv2d(32, 64, kernel_size=kernel_size, stride=stride, padding=padding),
            BatchNorm2d(64, momentum=momentum),
            LeakyReLU(negative_slope=negative_slope),
        )
        self.conv2 = Sequential(
            Conv2d(64, 128, kernel_size=kernel_size, stride=stride, padding=padding),
            BatchNorm2d(128, momentum=momentum),
            LeakyReLU(negative_slope=negative_slope),
        )
        self.dense1 = Sequential(
            Flatten(),
            Linear(32768, 1024),
            BatchNorm1d(1024, momentum=momentum),
            LeakyReLU(negative_slope=negative_slope),
        )

        ## the following take the same input from dense1
        self.dense_z_mu = Linear(1024, latent_dim)
        if alpha:
            self.dense_z_std = Sequential(
                Linear(1024, latent_dim),
                Softplus(),
            )
        self.optimizer = optimizer(self.parameters(), lr=learning_rate, betas=betas)

    def forward(self, x):
        x = self.conv0(x)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.dense1(x)
        z_mu = self.dense_z_mu(x)
        if not alpha:
            return z_mu
        z_std = self.dense_z_std(x)
        return cat((z_mu, z_std), dim=1)

        

In [ ]:
summarize(Encoder3, graph=False)

input size (3, 64, 64)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 64, 64]             128
         LeakyReLU-2           [-1, 32, 64, 64]               0
            Conv2d-3           [-1, 64, 32, 32]          51,264
       BatchNorm2d-4           [-1, 64, 32, 32]             128
         LeakyReLU-5           [-1, 64, 32, 32]               0
            Conv2d-6          [-1, 128, 16, 16]         204,928
       BatchNorm2d-7          [-1, 128, 16, 16]             256
         LeakyReLU-8          [-1, 128, 16, 16]               0
           Flatten-9                [-1, 32768]               0
           Linear-10                 [-1, 1024]      33,555,456
      BatchNorm1d-11                 [-1, 1024]           2,048
        LeakyReLU-12                 [-1, 1024]               0
           Linear-13                    [-1, 8]           8,200
           Linea

### Encoder 4
Conditional BatchNormed convolutional network with LeakyReLU

In [ ]:
class Encoder4(NeuralNetwork):
    """ convolutional network with BatchNorm and LeakyReLU """
    def __init__(self):
        super(Encoder4, self).__init__()
        kernel_size = 3
        stride = 2
        padding = (kernel_size - 1) // 2

        self.conv0 = Sequential(
            Conv2d(colors_dim, 16, kernel_size=1, stride=1),
            LeakyReLU(negative_slope=negative_slope),
        )
        self.conv1 = Sequential(
            Conv2d(16, 32, kernel_size=kernel_size, stride=stride, padding=padding),
            BatchNorm2d(32, momentum=momentum),
            LeakyReLU(negative_slope=negative_slope),
        )
        self.conv2 = Sequential(
            Conv2d(32, 64, kernel_size=kernel_size, stride=stride, padding=padding),
            BatchNorm2d(64, momentum=momentum),
            LeakyReLU(negative_slope=negative_slope),
        )
        self.conv3 = Sequential(
            Conv2d(64,128, kernel_size=kernel_size, stride=stride, padding=padding),
            BatchNorm2d(128, momentum=momentum),
            LeakyReLU(negative_slope=negative_slope),
            Flatten(), # next layer takes flat input with labels appended
        )
        self.dense1 = Sequential(
            Linear(8192+labels_dim, 1024),
            BatchNorm1d(1024, momentum=momentum),
            LeakyReLU(negative_slope=negative_slope)
        )

        ## the following take the same input from dense1
        self.dense_z_mu = Linear(1024, latent_dim)
        if alpha:
            self.dense_z_std = Sequential(
                Linear(1024, latent_dim),
                Softplus(),
            )
        self.optimizer = optimizer(self.parameters(), lr=learning_rate, betas=betas)

    def forward(self, images, labels):
        x = self.conv0(images)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = cat((x, labels), dim=1)
        x = self.dense1(x)
        z_mu = self.dense_z_mu(x)
        if not alpha:
            return z_mu
        z_std = self.dense_z_std(x)
        return z_mu, z_std



In [ ]:
net = Encoder4()

print(np.sum(np.prod(p.size()) for p in net.parameters()) == 8508656)
input_dummy = rand(3, *input_size)
label_dummy = rand(3, labels_dim)

net(input_dummy, label_dummy)

True


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  This is separate from the ipykernel package so we can avoid doing imports until


(tensor([[-0.0611, -0.2827, -0.0415,  0.4707, -0.4764,  0.3596,  0.8293,  0.1872],
         [ 0.4531, -0.3066, -0.5375, -0.2680, -0.3986, -0.2112, -0.7741, -0.6823],
         [-0.1973,  0.4452,  0.1442,  0.5908,  0.5428, -0.2410,  0.4880, -0.0132]],
        grad_fn=<AddmmBackward>),
 tensor([[0.5300, 0.5976, 1.1684, 0.9888, 0.6516, 0.5743, 0.7925, 0.7538],
         [0.7084, 0.6592, 0.4728, 0.6019, 0.5940, 0.5222, 0.6214, 0.3699],
         [0.4528, 0.7144, 0.5324, 0.3958, 0.2968, 0.5435, 0.4786, 0.6071]],
        grad_fn=<SoftplusBackward>))

## Decoder

In [ ]:
flat_dim = 8192
orig_dim = (128, 8, 8)

### Decoder 1
Inverse basic convolutional network

In [ ]:
class Decoder1(NeuralNetwork):
    """ reverse basic convolutional network """
    def __init__(self):
        super(Decoder1, self).__init__()
        kernel_size = 3
        stride = 2
        padding = (kernel_size - 1) // 2 
        self.dense1 = Sequential(
            Linear(latent_dim, flat_dim),
            BatchNorm1d(flat_dim),
            ReLU(),
            Reshape(*orig_dim)
        )
        self.conv1 = Sequential(
            ConvTranspose2d(orig_dim[0], 32, kernel_size=kernel_size, stride=stride, padding=padding, output_padding=padding),
            ReLU(),
        )
        self.conv2 = Sequential(
            ConvTranspose2d(32, 16, kernel_size=kernel_size, stride=stride, padding=padding, output_padding=padding),
            ReLU(),
        )
        self.conv3 = Sequential(
            ConvTranspose2d(16, 8, kernel_size=kernel_size, stride=stride, padding=padding, output_padding=padding),
            BatchNorm2d(8),
            ReLU(),
        )
        self.conv4 = Sequential(
            ConvTranspose2d(8, colors_dim, kernel_size=1, stride=1),
            Sigmoid(),
        )
        self.optimizer = optimizer(self.parameters(), lr=learning_rate, betas=betas)

    def forward(self, x):
        x = self.dense1(x)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        return x

In [ ]:
summarize(Decoder1, input_size=(latent_dim,))

input size (8,)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 8192]          73,728
       BatchNorm1d-2                 [-1, 8192]          16,384
              ReLU-3                 [-1, 8192]               0
           Reshape-4            [-1, 128, 8, 8]               0
   ConvTranspose2d-5           [-1, 32, 16, 16]          36,896
              ReLU-6           [-1, 32, 16, 16]               0
   ConvTranspose2d-7           [-1, 16, 32, 32]           4,624
              ReLU-8           [-1, 16, 32, 32]               0
   ConvTranspose2d-9            [-1, 8, 64, 64]           1,160
      BatchNorm2d-10            [-1, 8, 64, 64]              16
             ReLU-11            [-1, 8, 64, 64]               0
  ConvTranspose2d-12            [-1, 3, 64, 64]              27
          Sigmoid-13            [-1, 3, 64, 64]               0
Total params: 132,835
T

### Decoder 2
Inverse inception convolutional network

In [ ]:
class Decoder2(NeuralNetwork):
    """ reverse inception convolutional network """
    def __init__(self):
        super(Decoder2, self).__init__()
        kernel_size = 3
        stride = 2
        padding = (kernel_size - 1) // 2 
        self.dense1 = Sequential(
            Linear(latent_dim, flat_dim),
            BatchNorm1d(flat_dim),
            ReLU(),
            Reshape(*orig_dim)
        )
        # inception 1
        self.inc1_1 = Sequential(
            Conv2d(orig_dim[0], 16, kernel_size=1, stride=1),
            ReLU(),
        )
        self.inc1_2 = Sequential(
            Conv2d(orig_dim[0], 16, kernel_size=kernel_size, stride=1, padding=padding),
            ReLU(),
        )
        self.conv1 = Sequential(
            ConvTranspose2d(32, 32, kernel_size=kernel_size, stride=stride, padding=padding, output_padding=padding),
            ReLU(),
        )
        # inception 2
        self.inc2_1 = Sequential(
            Conv2d(32, 8, kernel_size=1, stride=1),
            ReLU(),
        )
        self.inc2_2 = Sequential(
            Conv2d(32, 8, kernel_size=kernel_size, stride=1, padding=padding),
            ReLU(),
        )
        self.conv2 = Sequential(
            ConvTranspose2d(16, 16, kernel_size=kernel_size, stride=stride, padding=padding, output_padding=padding),
            ReLU(),
        )
        # inception 3
        self.inc3_1 = Sequential(
            Conv2d(16, 4, kernel_size=1, stride=1),
            ReLU(),
        )
        self.inc3_2 = Sequential(
            Conv2d(16, 4, kernel_size=kernel_size, stride=1, padding=padding),
            ReLU(),
        )
        self.conv3 = Sequential(
            ConvTranspose2d(8, 8, kernel_size=kernel_size, stride=stride, padding=padding, output_padding=padding),
            BatchNorm2d(8),
            ReLU(),
        )
        # inception 4
        self.inc4_1 = Sequential(
            Conv2d(8, 2, kernel_size=1, stride=1),
            ReLU(),
        )
        self.inc4_2 = Sequential(
            Conv2d(8, 2, kernel_size=kernel_size, stride=1, padding=padding),
            ReLU(),
        )
        self.conv4 = Sequential(
            ConvTranspose2d(4, 4, kernel_size=kernel_size, stride=stride, padding=padding, output_padding=padding),
            Sigmoid(),
        )
        self.optimizer = optimizer(self.parameters(), lr=learning_rate, betas=betas)

    def forward(self, x):
        x = self.dense1(x)
        x1 = self.inc1_1(x)
        x2 = self.inc1_2(x)
        x = self.conv1(cat((x1, x2), dim=1))
        x1 = self.inc2_1(x)
        x2 = self.inc2_2(x)
        x = self.conv2(cat((x1, x2), dim=1))
        x1 = self.inc3_1(x)
        x2 = self.inc3_2(x)
        x = self.conv3(cat((x1, x2), dim=1))
        x1 = self.inc4_1(x)
        x2 = self.inc4_2(x)
        x = self.conv4(cat((x1, x2), dim=1))
        return x

In [ ]:
summarize(Decoder2, input_size=(latent_dim,))

input size (8,)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 8192]          73,728
       BatchNorm1d-2                 [-1, 8192]          16,384
              ReLU-3                 [-1, 8192]               0
           Reshape-4            [-1, 128, 8, 8]               0
            Conv2d-5             [-1, 16, 8, 8]           2,064
              ReLU-6             [-1, 16, 8, 8]               0
            Conv2d-7             [-1, 16, 8, 8]          18,448
              ReLU-8             [-1, 16, 8, 8]               0
   ConvTranspose2d-9           [-1, 32, 16, 16]           9,248
             ReLU-10           [-1, 32, 16, 16]               0
           Conv2d-11            [-1, 8, 16, 16]             264
             ReLU-12            [-1, 8, 16, 16]               0
           Conv2d-13            [-1, 8, 16, 16]           2,312
             ReLU-14   

### Decoder 3
Inverse BatchNormed convolutional network with LeakyReLU and kernel=5

In [ ]:
class Decoder3(NeuralNetwork):
    """ reverse convolutional network with kernel of 5"""
    def __init__(self):
        super(Decoder3, self).__init__()
        kernel_size = 5
        stride = 2
        padding = (kernel_size - 1) // 2 
        self.dense1 = Sequential(
            Linear(latent_dim, 1024),
            BatchNorm1d(1024, momentum=momentum),
            ReLU(),
        )
        self.dense2= Sequential(
            Linear(1024, 8*8*128),
            BatchNorm1d(8*8*128, momentum=momentum),
            ReLU(),
            Reshape(128, 8, 8)
        )
        self.conv1 = Sequential(
            ConvTranspose2d(128, 64, kernel_size=kernel_size, stride=stride, padding=padding, output_padding=1),
            BatchNorm2d(64, momentum=momentum),
            ReLU(),
        )
        self.conv2 = Sequential(
            ConvTranspose2d(64, 32, kernel_size=kernel_size, stride=stride, padding=padding, output_padding=1),
            BatchNorm2d(32, momentum=momentum),
            ReLU(),
        )
        self.conv3 = Sequential(
            ConvTranspose2d(32, 16, kernel_size=kernel_size, stride=stride, padding=padding, output_padding=1),
            BatchNorm2d(16, momentum=momentum),
            ReLU(),
        )
        self.conv4 = Sequential(
            ConvTranspose2d(16, colors_dim, kernel_size=1),
            Sigmoid()
        )
        self.optimizer = optimizer(self.parameters(), lr=learning_rate, betas=betas)

    def forward(self, x):
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        return x


In [ ]:
summarize(Decoder3, input_size=(latent_dim,))

input size (8,)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 1024]           9,216
       BatchNorm1d-2                 [-1, 1024]           2,048
              ReLU-3                 [-1, 1024]               0
            Linear-4                 [-1, 8192]       8,396,800
       BatchNorm1d-5                 [-1, 8192]          16,384
              ReLU-6                 [-1, 8192]               0
           Reshape-7            [-1, 128, 8, 8]               0
   ConvTranspose2d-8           [-1, 64, 16, 16]         204,864
       BatchNorm2d-9           [-1, 64, 16, 16]             128
             ReLU-10           [-1, 64, 16, 16]               0
  ConvTranspose2d-11           [-1, 32, 32, 32]          51,232
      BatchNorm2d-12           [-1, 32, 32, 32]              64
             ReLU-13           [-1, 32, 32, 32]               0
  ConvTranspose2d-14   

### Decoder 4

In [ ]:
class Decoder4(NeuralNetwork):
    """ reverse convolutional network with kernel of 5"""
    def __init__(self):
        super(Decoder4, self).__init__()
        kernel_size = 3
        stride = 2
        padding = (kernel_size - 1) // 2 
        self.dense1 = Sequential(
            Linear(latent_dim + labels_dim, 1024),
            BatchNorm1d(1024, momentum=momentum),
            ReLU(),
        )
        self.dense2 = Sequential(
            Linear(1024, flat_dim),
            BatchNorm1d(flat_dim, momentum=momentum),
            ReLU(),
            Reshape(*orig_dim),
        )
        self.conv1 = Sequential(
            ConvTranspose2d(orig_dim[0], 64, kernel_size=kernel_size, stride=stride, padding=padding, output_padding=padding),
            BatchNorm2d(64, momentum=momentum),
            ReLU(),
        )
        self.conv2 = Sequential(
            ConvTranspose2d(64, 32, kernel_size=kernel_size, stride=stride, padding=padding, output_padding=padding),
            BatchNorm2d(32, momentum=momentum),
            ReLU(),
        )
        self.conv3 = Sequential(
            ConvTranspose2d(32, 16, kernel_size=kernel_size, stride=stride, padding=padding, output_padding=padding),
            BatchNorm2d(16, momentum=momentum),
            ReLU(),
        )
        self.conv4 = Sequential(
            ConvTranspose2d(16, colors_dim, kernel_size=1, stride=1),
            Sigmoid(),
        )
        self.optimizer = optimizer(self.parameters(), lr=learning_rate, betas=betas)

    def forward(self, latent, labels):
        x = self.dense1(cat((latent, labels), dim=1))
        x = self.dense2(x)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        return x
        

In [ ]:
net = Decoder4()

print(np.sum(np.prod(p.size()) for p in net.parameters()) )
input_dummy = rand(3, latent_dim)
label_dummy = rand(3, labels_dim)

net(input_dummy, label_dummy).shape

8524675


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  This is separate from the ipykernel package so we can avoid doing imports until


torch.Size([3, 3, 64, 64])

# Sample

In [ ]:
# latent space
def gaussian_sampler(mu, sigma):
    batch = mu.shape[0]
    dim = mu.shape[1]
    return mu + sigma*torch.randn(batch, dim, requires_grad=True).cuda()

# Pipelines

In [ ]:
## all pipelines use the same networks
encoder = Encoder4().cuda()
try:
    encoder.load()
except:
    print("fresh encoder")
decoder = Decoder4().cuda()
try:
    decoder.load()
except:
    print("fresh decoder")
discriminator = Discriminator4().cuda()
try:
    discriminator.load()
except:
    print("fresh discriminator")


## CVAE
Encoder + Decoder with labels

In [ ]:
class VAE(NeuralNetwork):
    def __init__(self):
        super(VAE, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.sampler = gaussian_sampler

    def forward(self, images, labels):
        latent = self.encoder(images, labels)
        if alpha: ## in VAE mode, replace z by random sample
            latent = self.sampler(*latent)
        x_hat = self.decoder(latent,labels)
        return x_hat


In [ ]:
net = VAE()
images = rand(5,3,64,64).cuda()
labels = rand(5,3).cuda()
pred = net(images, labels)
pred.shape

torch.Size([5, 3, 64, 64])

## VAE-GAN
VAE + Discriminator

In [ ]:

class VAEGAN(NeuralNetwork):
    def __init__(self):
        super(VAEGAN, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.discriminator = discriminator
        self.sampler = gaussian_sampler

    def forward(self, images, labels):
        latent = self.encoder(images, labels)
        if alpha:
            latent = self.sampler(*latent)
        generated_images = self.decoder(latent,labels)

        output = self.discriminator(generated_images)
        return output # true/fake, labels, metric


In [ ]:
net = VAEGAN()
input = rand(3,3,64,64).cuda()
labels = rand(3,3).cuda()
pred = net(input, labels)
pred.shape, pred[:,0]

(torch.Size([3, 1028]),
 tensor([0.0064, 0.0347, 0.0162], device='cuda:0', grad_fn=<SelectBackward>))

# Loss function

In [ ]:
## pytorch has no categorical crossentropy for e. g. target = [0.3, 0.2, 0.5] instead of [0,0,1] or actually 2
## use the same loss as tensorflow: - sum target * log(prediction)
def categorical_crossentropy(target, prediction):
    """ calculate loss for one hot encoded labels with uncertain target """
    loss = - sum(target * log(prediction))
    return loss

mse = MSELoss()
bce = BCELoss()
cross_entropy = categorical_crossentropy


In [ ]:
def loss_reconstruction(image, generated_image):
    """ divergence of generated image from input image """
    return mse(image, generated_image) * image_dim**2

In [ ]:
def loss_kl(latent):
    """ divergence of recontstructed latent distribution from true distribution, assumed to be unit gaussian """
    loss = 1 + 2*log(latent[1]) - square(latent[0]) - square(latent[1])
    loss = -0.5 * sum(loss) #, axis=-1)
    return loss

In [ ]:
def loss_VAE(image, generated_image, latent_mean, latent_std):
    """ total loss of VAE """
    ## !! care for correct handling if latent in loss_kl
    loss = loss_reconstruction(image, generated_image)
    if alpha:
        loss += loss_kl(latent_mean, latent_std)
    return mean(loss)

In [ ]:
def loss_adversarial(target, prediction):
    """ divergence of classification of whether image is part of sample distribution """
    return bce(prediction, target)

In [ ]:
def loss_class(target, prediction):
    """ divergence of classification of subclasses in sample distribution """
    return cross_entropy(target, prediction)

In [ ]:
def loss_metric(target, prediction):
    """ divergence of internal metric """
    return mse(target, prediction)

In [ ]:
def loss_GAN(target, prediction, image, generated_image, latent):
    """ total loss of GAN
    
    target and prediction contain (axis=1)
        0               ; binary classification
        1:labels_dim    ; label classification
        labels_dim+1:-1 ; metric
    """
    loss = loss_adversarial(target[:,0], prediction[:,0])
    loss += delta * loss_class(target[:,1:1+labels_dim], prediction[:,1:1+labels_dim])
    loss += gamma * loss_metric(target[:,2+labels_dim:], prediction[:,2+labels_dim:])
    loss += zeta * loss_reconstruction(image, generated_image)
    if alpha:
        loss += beta * loss_kl(latent)
    return mean(loss)

In [ ]:
def loss_discriminator(target, prediction):
    """ total loss of discriminator """
    loss = loss_adversarial(target[:,0], prediction[:,0])
    loss += delta * loss_class(target[:,1:1+labels_dim], prediction[:,1:1+labels_dim])
    return mean(loss)

# Data Set

In [ ]:
# galaxy images
x_train = np.load(file_galaxy_images)
x_train = x_train/255.0 ## rescale to 0<x<1
x_train = np.rollaxis(x_train, -1, 1)
N_samples = x_train.shape[0]
x_train = from_numpy(x_train).cuda()
x_train.shape

torch.Size([61578, 3, 64, 64])

In [ ]:
# hierarchical galaxy labels
df_galaxy_labels =  pd.read_csv(file_galaxy_labels)
display(df_galaxy_labels.head())
labels_train = df_galaxy_labels[df_galaxy_labels.columns[1:4]].values
labels_train = from_numpy(labels_train).float().cuda()
labels_train.shape

,GalaxyID,Class1.1,Class1.2,Class1.3,Class2.1,Class2.2,Class3.1,Class3.2,Class4.1,Class4.2,Class5.1,Class5.2,Class5.3,Class5.4,Class6.1,Class6.2,Class7.1,Class7.2,Class7.3,Class8.1,Class8.2,Class8.3,Class8.4,Class8.5,Class8.6,Class8.7,Class9.1,Class9.2,Class9.3,Class10.1,Class10.2,Class10.3,Class11.1,Class11.2,Class11.3,Class11.4,Class11.5,Class11.6
0,100008,0.383147,0.616853,0.000000,0.000000,0.616853,0.038452,0.578401,0.418398,0.198455,0.0,0.104752,0.512101,0.000000,0.054453,0.945547,0.201463,0.181684,0.000000,0.0,0.027226,0.000000,0.027226,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.279952,0.138445,0.000000,0.000000,0.092886,0.0,0.0,0.0,0.325512
1,100023,0.327001,0.663777,0.009222,0.031178,0.632599,0.467370,0.165229,0.591328,0.041271,0.0,0.236781,0.160941,0.234877,0.189149,0.810851,0.000000,0.135082,0.191919,0.0,0.000000,0.140353,0.000000,0.048796,0.000000,0.0,0.012414,0.0,0.018764,0.000000,0.131378,0.459950,0.000000,0.591328,0.0,0.0,0.0,0.000000
2,100053,0.765717,0.177352,0.056931,0.000000,0.177352,0.000000,0.177352,0.000000,0.177352,0.0,0.117790,0.059562,0.000000,0.000000,1.000000,0.000000,0.741864,0.023853,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000
3,100078,0.693377,0.238564,0.068059,0.000000,0.238564,0.109493,0.129071,0.189098,0.049466,0.0,0.000000,0.113284,0.125280,0.320398,0.679602,0.408599,0.284778,0.000000,0.0,0.000000,0.096119,0.096119,0.000000,0.128159,0.0,0.000000,0.0,0.000000,0.094549,0.000000,0.094549,0.189098,0.000000,0.0,0.0,0.0,0.000000
4,100090,0.933839,0.000000,0.066161,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.029383,0.970617,0.494587,0.439252,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.029383,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000


torch.Size([61578, 3])

In [ ]:
## completely random batch each next()
def make_training_sample_generator(batch_size):
    while True:
        idx = np.random.choice(range(N_samples), size=batch_size, replace=False)
        yield x_train[idx], labels_train[idx]

## randomized batches with each sample chosen at most once per epoch
def make_training_sample_generator(batch_size):
    indices = np.random.permutation(N_samples)
    for i in range(int(N_samples/batch_size)):
        idx = return_batch(indices, i, batch_size)
        yield x_train[idx], labels_train[idx]

# Training

In [ ]:
epochs = 20
batch_size = 128
steps = N_samples // batch_size
save_interval = 200

discriminator_losses = []
discriminator_losses_real = []
discriminator_losses_fake = []
generator_losses = []

valid = torch.ones((batch_size,1)).cuda()
fake = torch.zeros((batch_size,1)).cuda()

vae = VAE()
vaegan = VAEGAN()

iteration = 0
epoch = 0
step = 0



In [ ]:
def training_step():
    global iteration
    images, labels = next(sample_generator)

    # -------------------
    # Train Discriminator
    # -------------------
    vae.train(False)
    discriminator.train(True)
    discriminator.zero_grad()

    generated_images = vae(images, labels)
    target_real = cat((valid,labels), dim=1)
    prediction_real = discriminator(images)[:,:1+labels_dim]
    target_fake = cat((fake, labels), dim=1)
    prediction_fake = discriminator(generated_images)[:,:1+labels_dim]

    d_loss_real = loss_discriminator(target_real, prediction_real)
    d_loss_fake = loss_discriminator(target_fake, prediction_fake)
    d_loss = 0.5 * add(d_loss_fake, d_loss_real)
    discriminator_losses.append(d_loss)
    discriminator_losses_fake.append(d_loss_fake)
    discriminator_losses_real.append(d_loss_real)

    d_loss_real.backward()
    d_loss_fake.backward()
    discriminator.optimizer.step()

    # ---------------
    # Train Generator
    # ---------------
    vae.train(True)
    discriminator.train(False)
    encoder.zero_grad()
    decoder.zero_grad()

    generated_images = vae(images, labels)
    target = discriminator(images)
    target[:,0] = 1
    target[:,1:1+labels_dim] = labels
    target = target.detach()
    prediction = discriminator(generated_images)
    latent = encoder(images, labels)

    g_loss = loss_GAN(target, prediction, images, generated_images, latent)
    g_loss.backward()
    encoder.optimizer.step()
    decoder.optimizer.step()
    generator_losses.append(g_loss)


    iteration += 1

    print(f"iteration {iteration}, epoch {epoch+1}, batch {step+1}/{steps}," + \
          f"disc_loss {d_loss:.5}, (real {d_loss_real:.5}, fake {d_loss_fake:.5} ) gen_loss {g_loss:.5}")

    if not iteration % save_interval:
        write_generated_galaxy_images_iteration(iteration=iteration, images=generated_images.detach().cpu().numpy())


In [ ]:
while epoch < epochs:
    sample_generator = make_training_sample_generator(batch_size)
    step = 0
    while step < steps:
        training_step()
        step += 1
    epoch += 1

    # save a plot of the costs
    plt.clf()
    plt.plot(discriminator_losses, label='discriminator cost')
    plt.plot(generator_losses, label='generator cost')
    plt.plot(discriminator_losses_fake, label='discriminator cost fake', linestyle=":")
    plt.plot(discriminator_losses_real, label='discriminator cost real', linestyle="-.")
    plt.yscale("log")
    plt.legend()
    plt.savefig(folder_results+"cost_vs_iteration.png")
    plt.close()

    decoder.save()
    encoder.save()
    discriminator.save()

iteration 201, epoch 1, batch 1/481,disc_loss 79.212, (real 83.769, fake 74.655 ) gen_loss 310.68
iteration 202, epoch 1, batch 2/481,disc_loss 79.836, (real 86.036, fake 73.636 ) gen_loss 293.07
iteration 203, epoch 1, batch 3/481,disc_loss 78.791, (real 82.5, fake 75.083 ) gen_loss 304.12
iteration 204, epoch 1, batch 4/481,disc_loss 80.133, (real 84.288, fake 75.977 ) gen_loss 297.68
iteration 205, epoch 1, batch 5/481,disc_loss 78.573, (real 84.24, fake 72.906 ) gen_loss 321.8
iteration 206, epoch 1, batch 6/481,disc_loss 85.417, (real 89.631, fake 81.204 ) gen_loss 311.72
iteration 207, epoch 1, batch 7/481,disc_loss 84.245, (real 90.055, fake 78.434 ) gen_loss 304.15
iteration 208, epoch 1, batch 8/481,disc_loss 82.226, (real 88.576, fake 75.876 ) gen_loss 280.0
iteration 209, epoch 1, batch 9/481,disc_loss 81.437, (real 89.204, fake 73.671 ) gen_loss 311.29
iteration 210, epoch 1, batch 10/481,disc_loss 88.37, (real 92.969, fake 83.772 ) gen_loss 300.66
iteration 211, epoch 1, b

iteration 401, epoch 1, batch 201/481,disc_loss 83.215, (real 89.635, fake 76.794 ) gen_loss 333.74
iteration 402, epoch 1, batch 202/481,disc_loss 80.619, (real 86.054, fake 75.184 ) gen_loss 297.35
iteration 403, epoch 1, batch 203/481,disc_loss 78.931, (real 84.339, fake 73.523 ) gen_loss 337.68
iteration 404, epoch 1, batch 204/481,disc_loss 79.774, (real 86.124, fake 73.425 ) gen_loss 317.63
iteration 405, epoch 1, batch 205/481,disc_loss 82.205, (real 86.413, fake 77.998 ) gen_loss 343.05
iteration 406, epoch 1, batch 206/481,disc_loss 81.871, (real 85.836, fake 77.906 ) gen_loss 340.06
iteration 407, epoch 1, batch 207/481,disc_loss 80.096, (real 83.302, fake 76.891 ) gen_loss 376.77
iteration 408, epoch 1, batch 208/481,disc_loss 78.384, (real 83.798, fake 72.969 ) gen_loss 356.34
iteration 409, epoch 1, batch 209/481,disc_loss 82.047, (real 86.648, fake 77.447 ) gen_loss 338.99
iteration 410, epoch 1, batch 210/481,disc_loss 83.818, (real 89.163, fake 78.473 ) gen_loss 328.96


iteration 601, epoch 1, batch 401/481,disc_loss 79.434, (real 84.561, fake 74.307 ) gen_loss 377.52
iteration 602, epoch 1, batch 402/481,disc_loss 83.344, (real 88.337, fake 78.35 ) gen_loss 405.02
iteration 603, epoch 1, batch 403/481,disc_loss 82.798, (real 87.437, fake 78.158 ) gen_loss 431.02
iteration 604, epoch 1, batch 404/481,disc_loss 82.388, (real 87.791, fake 76.985 ) gen_loss 390.67
iteration 605, epoch 1, batch 405/481,disc_loss 79.564, (real 84.318, fake 74.809 ) gen_loss 397.57
iteration 606, epoch 1, batch 406/481,disc_loss 78.93, (real 83.246, fake 74.614 ) gen_loss 393.12
iteration 607, epoch 1, batch 407/481,disc_loss 80.816, (real 85.331, fake 76.302 ) gen_loss 366.89
iteration 608, epoch 1, batch 408/481,disc_loss 80.172, (real 83.612, fake 76.733 ) gen_loss 379.01
iteration 609, epoch 1, batch 409/481,disc_loss 81.431, (real 86.369, fake 76.493 ) gen_loss 397.59
iteration 610, epoch 1, batch 410/481,disc_loss 84.223, (real 88.413, fake 80.034 ) gen_loss 425.03
it

iteration 801, epoch 2, batch 120/481,disc_loss 80.923, (real 84.099, fake 77.747 ) gen_loss 459.1
iteration 802, epoch 2, batch 121/481,disc_loss 81.536, (real 85.798, fake 77.274 ) gen_loss 402.47
iteration 803, epoch 2, batch 122/481,disc_loss 80.585, (real 85.341, fake 75.83 ) gen_loss 404.02
iteration 804, epoch 2, batch 123/481,disc_loss 80.744, (real 86.494, fake 74.995 ) gen_loss 376.93
iteration 805, epoch 2, batch 124/481,disc_loss 85.042, (real 88.738, fake 81.347 ) gen_loss 365.34
iteration 806, epoch 2, batch 125/481,disc_loss 76.649, (real 80.809, fake 72.488 ) gen_loss 388.1
iteration 807, epoch 2, batch 126/481,disc_loss 81.08, (real 85.073, fake 77.088 ) gen_loss 412.52
iteration 808, epoch 2, batch 127/481,disc_loss 82.017, (real 85.34, fake 78.695 ) gen_loss 391.57
iteration 809, epoch 2, batch 128/481,disc_loss 78.77, (real 83.825, fake 73.715 ) gen_loss 426.15
iteration 810, epoch 2, batch 129/481,disc_loss 86.185, (real 90.5, fake 81.87 ) gen_loss 410.99
iteration

iteration 1001, epoch 2, batch 320/481,disc_loss 79.649, (real 84.291, fake 75.006 ) gen_loss 446.46
iteration 1002, epoch 2, batch 321/481,disc_loss 82.232, (real 86.276, fake 78.189 ) gen_loss 449.26
iteration 1003, epoch 2, batch 322/481,disc_loss 82.469, (real 88.029, fake 76.909 ) gen_loss 421.46
iteration 1004, epoch 2, batch 323/481,disc_loss 84.709, (real 88.92, fake 80.497 ) gen_loss 435.53
iteration 1005, epoch 2, batch 324/481,disc_loss 78.074, (real 82.958, fake 73.191 ) gen_loss 439.35
iteration 1006, epoch 2, batch 325/481,disc_loss 83.443, (real 87.689, fake 79.197 ) gen_loss 428.1
iteration 1007, epoch 2, batch 326/481,disc_loss 80.404, (real 84.631, fake 76.177 ) gen_loss 442.86
iteration 1008, epoch 2, batch 327/481,disc_loss 77.998, (real 83.399, fake 72.597 ) gen_loss 423.13
iteration 1009, epoch 2, batch 328/481,disc_loss 81.236, (real 86.254, fake 76.218 ) gen_loss 457.93
iteration 1010, epoch 2, batch 329/481,disc_loss 79.275, (real 84.045, fake 74.505 ) gen_loss

...

iteration 9000, epoch 19, batch 142/481,disc_loss 74.525, (real 76.46, fake 72.589 ) gen_loss 865.89
iteration 9001, epoch 19, batch 143/481,disc_loss 79.084, (real 81.62, fake 76.548 ) gen_loss 823.42
iteration 9002, epoch 19, batch 144/481,disc_loss 78.211, (real 80.44, fake 75.982 ) gen_loss 906.33
iteration 9003, epoch 19, batch 145/481,disc_loss 76.117, (real 78.529, fake 73.705 ) gen_loss 879.05
iteration 9004, epoch 19, batch 146/481,disc_loss 72.255, (real 74.004, fake 70.506 ) gen_loss 972.39
iteration 9005, epoch 19, batch 147/481,disc_loss 77.378, (real 79.663, fake 75.092 ) gen_loss 868.87
iteration 9006, epoch 19, batch 148/481,disc_loss 77.963, (real 80.909, fake 75.017 ) gen_loss 962.41
iteration 9007, epoch 19, batch 149/481,disc_loss 73.631, (real 75.716, fake 71.546 ) gen_loss 1042.6
iteration 9008, epoch 19, batch 150/481,disc_loss 74.75, (real 77.088, fake 72.413 ) gen_loss 857.46
iteration 9009, epoch 19, batch 151/481,disc_loss 76.592, (real 78.007, fake 75.177 ) 

iteration 9200, epoch 19, batch 342/481,disc_loss 75.873, (real 77.626, fake 74.12 ) gen_loss 889.87
iteration 9201, epoch 19, batch 343/481,disc_loss 76.649, (real 78.284, fake 75.014 ) gen_loss 863.25
iteration 9202, epoch 19, batch 344/481,disc_loss 79.083, (real 81.525, fake 76.641 ) gen_loss 831.2
iteration 9203, epoch 19, batch 345/481,disc_loss 79.564, (real 81.617, fake 77.511 ) gen_loss 894.9
iteration 9204, epoch 19, batch 346/481,disc_loss 76.307, (real 79.249, fake 73.364 ) gen_loss 895.33
iteration 9205, epoch 19, batch 347/481,disc_loss 80.945, (real 83.399, fake 78.49 ) gen_loss 936.4
iteration 9206, epoch 19, batch 348/481,disc_loss 77.435, (real 79.612, fake 75.259 ) gen_loss 889.13
iteration 9207, epoch 19, batch 349/481,disc_loss 78.867, (real 81.508, fake 76.227 ) gen_loss 863.76
iteration 9208, epoch 19, batch 350/481,disc_loss 74.769, (real 77.543, fake 71.994 ) gen_loss 865.02
iteration 9209, epoch 19, batch 351/481,disc_loss 75.819, (real 78.077, fake 73.561 ) g

iteration 9400, epoch 20, batch 61/481,disc_loss 73.855, (real 76.283, fake 71.427 ) gen_loss 890.28
iteration 9401, epoch 20, batch 62/481,disc_loss 74.517, (real 76.946, fake 72.088 ) gen_loss 867.81
iteration 9402, epoch 20, batch 63/481,disc_loss 75.636, (real 78.207, fake 73.064 ) gen_loss 833.18
iteration 9403, epoch 20, batch 64/481,disc_loss 74.546, (real 76.141, fake 72.95 ) gen_loss 883.88
iteration 9404, epoch 20, batch 65/481,disc_loss 74.491, (real 76.834, fake 72.147 ) gen_loss 867.05
iteration 9405, epoch 20, batch 66/481,disc_loss 75.198, (real 76.983, fake 73.414 ) gen_loss 879.56
iteration 9406, epoch 20, batch 67/481,disc_loss 69.784, (real 71.34, fake 68.229 ) gen_loss 954.62
iteration 9407, epoch 20, batch 68/481,disc_loss 70.806, (real 72.647, fake 68.965 ) gen_loss 933.1
iteration 9408, epoch 20, batch 69/481,disc_loss 75.397, (real 77.385, fake 73.409 ) gen_loss 939.11
iteration 9409, epoch 20, batch 70/481,disc_loss 76.832, (real 79.193, fake 74.471 ) gen_loss 

iteration 9600, epoch 20, batch 261/481,disc_loss 71.932, (real 74.641, fake 69.223 ) gen_loss 936.03
iteration 9601, epoch 20, batch 262/481,disc_loss 77.773, (real 79.795, fake 75.751 ) gen_loss 929.1
iteration 9602, epoch 20, batch 263/481,disc_loss 74.508, (real 76.602, fake 72.414 ) gen_loss 1035.3
iteration 9603, epoch 20, batch 264/481,disc_loss 78.772, (real 80.806, fake 76.738 ) gen_loss 838.03
iteration 9604, epoch 20, batch 265/481,disc_loss 75.162, (real 77.097, fake 73.226 ) gen_loss 917.74
iteration 9605, epoch 20, batch 266/481,disc_loss 80.836, (real 82.995, fake 78.677 ) gen_loss 874.63
iteration 9606, epoch 20, batch 267/481,disc_loss 78.99, (real 81.17, fake 76.81 ) gen_loss 932.99
iteration 9607, epoch 20, batch 268/481,disc_loss 77.885, (real 80.951, fake 74.82 ) gen_loss 1030.7
iteration 9608, epoch 20, batch 269/481,disc_loss 78.957, (real 81.821, fake 76.094 ) gen_loss 925.53
iteration 9609, epoch 20, batch 270/481,disc_loss 77.397, (real 79.859, fake 74.934 ) g

iteration 9800, epoch 20, batch 461/481,disc_loss 73.625, (real 75.526, fake 71.724 ) gen_loss 983.33
iteration 9801, epoch 20, batch 462/481,disc_loss 74.727, (real 76.174, fake 73.28 ) gen_loss 874.32
iteration 9802, epoch 20, batch 463/481,disc_loss 77.303, (real 79.635, fake 74.972 ) gen_loss 849.53
iteration 9803, epoch 20, batch 464/481,disc_loss 76.012, (real 79.026, fake 72.998 ) gen_loss 976.8
iteration 9804, epoch 20, batch 465/481,disc_loss 79.658, (real 82.557, fake 76.76 ) gen_loss 1019.5
iteration 9805, epoch 20, batch 466/481,disc_loss 75.188, (real 77.984, fake 72.391 ) gen_loss 887.47
iteration 9806, epoch 20, batch 467/481,disc_loss 74.452, (real 76.409, fake 72.494 ) gen_loss 824.73
iteration 9807, epoch 20, batch 468/481,disc_loss 76.04, (real 78.108, fake 73.973 ) gen_loss 877.66
iteration 9808, epoch 20, batch 469/481,disc_loss 81.695, (real 85.735, fake 77.654 ) gen_loss 819.83
iteration 9809, epoch 20, batch 470/481,disc_loss 74.516, (real 77.695, fake 71.336 ) 

# Testing

In [ ]:
!pip install line_profiler
%load_ext line_profiler
%lprun -f training_step training_step()

In [ ]:
labels_train.dtype

torch.float64